In [1]:
import torch
import time
import math
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

In [2]:
def init_model():
    model = AutoModelForCausalLM.from_pretrained(
        "/data/tds/ChatGLM3/chatglm3-6b",
        torch_dtype=torch.float16,
        device_map="cuda:0",
        trust_remote_code=True
    )
    tokenizer = AutoTokenizer.from_pretrained(
        "/data/tds/ChatGLM3/chatglm3-6b",
        use_fast=False,
        trust_remote_code=True
    )
    return model, tokenizer


model, tokenizer = init_model()
model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in_

In [62]:
def gen_prompt(user_question):
    prompt = f"""
请根据给定的句子，判断其意图是否为写一段营销文案：

输入句子：
{user_question}

判断逻辑：
如果输入句子中包含以下所有特征，则判断为写营销文案的请求，请输出“是”：
请求生成文案：句子中包含“帮我写个文案”、“生成文案”等明确的请求。
特定风格要求：句子中指定了特定的文案风格，如“小红书风格”、“微博风格”等。
产品或服务信息：句子中明确提及了某个产品或服务，包括品牌、名称、特点等。
营销目标：句子的主要目标是为了推广、宣传或销售某个产品或服务。

如果输入句子不满足以上任一特征，则判断为非营销文案任务，请输出“否”。

输出：
如果句子是一个写营销文案的请求，请输出“是”。如果句子是一个非营销文案任务，请输出“否”。记住，你只能输出“是”或者“否”，请严格遵守输出规则。


"""
    return prompt
#prompt = "水的化学式是什么"
prompt = "康萃乐Culturelle益生菌为什么适合减肥？"
#prompt = "帮我写个文案，小红书风格的，帕兰朵品牌的内衣，小胸聚拢特点的"
response,history = model.chat(tokenizer, gen_prompt(prompt), history=[])
print(gen_prompt(prompt),response)


请根据给定的句子，判断其意图是否为写一段营销文案：

输入句子：
康萃乐Culturelle益生菌为什么适合减肥？

判断逻辑：
如果输入句子中包含以下特征，则判断为写营销文案的请求：

请求生成文案：句子中包含“帮我写个文案”、“生成文案”等明确的请求。
特定风格要求：句子中指定了特定的文案风格，如“小红书风格”、“微博风格”等。
营销目标：句子的主要目标是为了推广、宣传或销售某个产品或服务。
如果输入句子不满足以上特征，则判断为非营销文案任务。

输出：
如果句子是一个严格的请求写营销文案的请求，请输出“是”。如果句子不是一个严格的请求写营销文案的请求，请输出“否”。记住，你只能输出“是”或者“否”，请严格遵守输出规则。
 是


In [9]:
def inplace_prompt(df,old_part,newstr):
# 使用 applymap() 函数遍历 DataFrame 中的每个元素，并应用字符串替换方法  
    df = df.applymap(lambda x: x.replace(old_part, newstr) if isinstance(x, str) else x)  
    return df

In [17]:
df = pd.read_excel("chatglm_NR.xlsx")
old_str = """json格式为{"产品品类": ,"产品品牌": ,"产品特点": }"""
replace_str = """
json格式为{"产品品类":,"产品品牌": ,"产品特点": }，注意：json格式不能改变，只用填入对应信息即可。
输出例子：{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
{"产品品类": "美容仪", "产品品牌": "日立（HITACHI）", "产品特点": "提拉紧致,嫩肤淡纹,适合新手"}
{"产品品类": "益生菌", "产品品牌": "君乐宝", "产品特点": "能够改善消化功能、促进肠道蠕动、增加粪便量、改善便秘"}
"""
df = inplace_prompt(df,old_str,replace_str)
print(df.iloc[0,0])

请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，
json格式为{"产品品类":,"产品品牌": ,"产品特点": }，注意：json格式不能改变，只用填入对应信息即可。
输出例子：{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
{"产品品类": "美容仪", "产品品牌": "日立（HITACHI）", "产品特点": "提拉紧致,嫩肤淡纹,适合新手"}
{"产品品类": "益生菌", "产品品牌": "君乐宝", "产品特点": "能够改善消化功能、促进肠道蠕动、增加粪便量、改善便秘"}

帮我写个文案，小红书风格的，帕兰朵品牌的内衣，小胸聚拢特点的


In [20]:
for i in range(len(df)):
    history = []
    if not isinstance(df.iloc[i,1], str): #单轮对话
        response,history = model.chat(tokenizer, df.iloc[i,0], history=[])
        print(i,df.iloc[i,0],response)
        df.iloc[i,3] = response
    elif not isinstance(df.iloc[i,2], str): #两轮对话
        response , history = model.chat(tokenizer, df.iloc[i,0], history=[])
        print(i,df.iloc[i,0],response)
        df.iloc[i,3] = response
        response,history = model.chat(tokenizer, df.iloc[i,1], history)
        print(i,df.iloc[i,1],response)
        df.iloc[i,4] = response
    else: #三轮对话
        response , history = model.chat(tokenizer, df.iloc[i,0], history=[])
        print(i,df.iloc[i,0],response)
        df.iloc[i,3] = response
        response , history = model.chat(tokenizer, df.iloc[i,1], history)
        print(i,df.iloc[i,1],response)
        df.iloc[i,4] = response
        response,history = model.chat(tokenizer, df.iloc[i,2], history)
        print(i,df.iloc[i,2],response)
        df.iloc[i,5] = response
df.to_excel("chatglm_NR_output.xlsx")

0 请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，
json格式为{"产品品类":,"产品品牌": ,"产品特点": }，注意：json格式不能改变，只用填入对应信息即可。
输出例子：{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
{"产品品类": "美容仪", "产品品牌": "日立（HITACHI）", "产品特点": "提拉紧致,嫩肤淡纹,适合新手"}
{"产品品类": "益生菌", "产品品牌": "君乐宝", "产品特点": "能够改善消化功能、促进肠道蠕动、增加粪便量、改善便秘"}

帮我写个文案，小红书风格的，帕兰朵品牌的内衣，小胸聚拢特点的 {"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
1 请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，
json格式为{"产品品类":,"产品品牌": ,"产品特点": }，注意：json格式不能改变，只用填入对应信息即可。
输出例子：{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
{"产品品类": "美容仪", "产品品牌": "日立（HITACHI）", "产品特点": "提拉紧致,嫩肤淡纹,适合新手"}
{"产品品类": "益生菌", "产品品牌": "君乐宝", "产品特点": "能够改善消化功能、促进肠道蠕动、增加粪便量、改善便秘"}

帮我写个文案，小红书风格的，TriPollar品牌的美容仪，主打平价护肤、亲测好用 {"产品品类": "美容仪", "产品品牌": "TriPollar", "产品特点": "平价护肤,亲测好用"}
2 请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，
json格式为{"产品品类":,"产品品牌": ,"产品特点": }，注意：json格式不能改变，只用填入对应信息即可。
输出例子：{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
{"产品品类": "美容仪", "产品品牌": "日立（HITACHI）"

In [37]:
prompt="""
需要完成以下两个任务：
任务一：判断“养乐多里面有没有益生菌”是否为生成文案任务，如果是，继续完成任务二，否则输出{}
任务二：请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，
json格式为{"产品品类":,"产品品牌": ,"产品特点": }，注意：json格式不能改变，只用填入对应信息即可。
输出例子：{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
{"产品品类": "美容仪", "产品品牌": "日立（HITACHI）", "产品特点": "提拉紧致,嫩肤淡纹,适合新手"}
{"产品品类": "益生菌", "产品品牌": "养乐多", "产品特点": "能够改善消化功能"}

养乐多里面有没有益生菌

"""
response,history = model.chat(tokenizer, prompt, history=[])
print(prompt,response)


需要完成以下两个任务：
任务一：判断“养乐多里面有没有益生菌”是否为生成文案任务，如果是，继续完成任务二，否则输出{}
任务二：请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，
json格式为{"产品品类":,"产品品牌": ,"产品特点": }，注意：json格式不能改变，只用填入对应信息即可。
输出例子：{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢"}
{"产品品类": "美容仪", "产品品牌": "日立（HITACHI）", "产品特点": "提拉紧致,嫩肤淡纹,适合新手"}
{"产品品类": "益生菌", "产品品牌": "养乐多", "产品特点": "能够改善消化功能"}

养乐多里面有没有益生菌

 任务一：是生成文案任务

任务二：
{"产品品类": "益生菌", "产品品牌": "养乐多", "产品特点": "能够改善消化功能"}
